In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy import stats
from PIL import Image, ImageTk,ImageDraw, ImageFont
import os
import sys
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

In [2]:
# Create a black image, a window
img = np.zeros((300,512,3), np.uint8)
cv2.namedWindow('image')


# 学習に用いる縮小画像のサイズ
sw = 64
sh = 64

#コインの認識用パラメータ（HチャンネルとSチャンネルとを二値化するための条件）
hmin = 34
hmax = 115 
smin = 30

#rectangleの大きさ
min_size=900
max_size=5000



def getImageVector(img):
    # 白い領域(ピクセル値が0でない領域)の座標を集める
    nonzero = cv2.findNonZero(img)
    # その領域を囲う四角形の座標と大きさを取得
    xx, yy, ww, hh = cv2.boundingRect(nonzero)
    # 白い領域を含む最小の矩形領域を取得
    img_nonzero = img[yy:yy+hh, xx:xx+ww]
    # 白い領域を(sw, sh)サイズに縮小するための準備
    img_small = np.zeros((sh, sw), dtype=np.uint8)
    # 画像のアスペクト比を保ったまま、白い領域を縮小してimg_smallにコピーする
    if 1*hh < ww*1 and hh > 0:
        htmp = int(sw*hh/ww)
        if htmp>0:
            img_small_tmp = cv2.resize(img_nonzero, (sw, htmp), interpolation=cv2.INTER_LINEAR)
            img_small[(sh-htmp)//2:(sh-htmp)//2+htmp, 0:sw] = img_small_tmp
    elif 1*hh >= ww*1 and ww > 0:
        wtmp = int(sh*ww/hh)
        if wtmp>0:
            img_small_tmp = cv2.resize(img_nonzero, (wtmp, sh), interpolation=cv2.INTER_LINEAR)
            img_small[0:sh, (sw-wtmp)//2:(sw-wtmp)//2+wtmp] = img_small_tmp
    # 0...1の範囲にスケーリングしてからリターンする
    return np.array([img_small.ravel()/255.])

# VideoCapture オブジェクトを取得します
capture = cv2.VideoCapture(0)
capture.set(5, 15)   # FPS

i=0

In [3]:
tf.reset_default_graph()
net=input_data(shape=[None,64,64,1])
net=conv_2d(net,64,5,activation='relu')
net=max_pool_2d(net,7)
net=conv_2d(net,128,5,activation='relu')
net=max_pool_2d(net,7)
net=fully_connected(net,256,activation='relu')
net=dropout(net,0.5)

net=tflearn.fully_connected(net,7,activation='softmax')
net=tflearn.regression(net,optimizer='sgd',learning_rate=0.1,loss='categorical_crossentropy')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
model=tflearn.DNN(net)

In [5]:
model.load('./rec_coin.tflearn')

INFO:tensorflow:Restoring parameters from C:\Anaconda3\envs\tensol\rec_coin.tflearn


In [ ]:
while(True):
	ret,frame = capture.read()
	# 映像データをHSV形式に変換
	hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	# HSV形式からHチャンネルとSチャンネルの画像を得る
	hsv_channels = cv2.split(hsv)
	h_channel = hsv_channels[0]
	s_channel = hsv_channels[1]

	# Hチャンネルを平滑化
	h_binary_ = cv2.GaussianBlur(h_channel, (5,5), 0)

	# Hチャンネルの二値化画像を作成
	# hmin～hmaxの範囲を255（白）に、それ以外を0（黒）に
	ret,h_binary = cv2.threshold(h_binary_, hmax, 255, cv2.THRESH_TOZERO_INV)
	ret,h_binary = cv2.threshold(h_binary, hmin, 255, cv2.THRESH_BINARY)
	
	binary=cv2.bitwise_not(h_binary)
	
	_,contours,_ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	
	img=np.array(frame)
	prediction_count=0
	result=[]
	coin_cnt=0
	coin_1_cnt=0
	coin_5_cnt=0
	coin_10_cnt=0
	coin_50_cnt=0
	coin_100_cnt=0
	coin_500_cnt=0
	sum=0
	
	for c in contours:
		if (cv2.contourArea(c) < min_size or cv2.contourArea(c) >max_size):
            		continue
		x,y,w,h = cv2.boundingRect(c)
		cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
		img = cv2.cvtColor(frame[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
		img_vector = getImageVector(img)
		img_r = np.reshape(img_vector, (1,64,64,1))
		prediction=np.array(model.predict(img_r).argmax(axis=1))
		if(prediction==6):
			continue
		coin_cnt=coin_cnt+1
		if(prediction==0):
			coin_1_cnt=coin_1_cnt+1
			sum=sum+1
			cv2.putText(frame, '1yen', (x,y-2), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 1, cv2.LINE_AA)
		elif(prediction==1):
			coin_5_cnt=coin_5_cnt+1
			sum=sum+5
			cv2.putText(frame, '5yen', (x,y-2), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 1, cv2.LINE_AA)
		elif(prediction==2):
			coin_10_cnt=coin_10_cnt+1
			sum=sum+10
			cv2.putText(frame, '10yen', (x,y-2), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 1, cv2.LINE_AA)
		elif(prediction==3):
			coin_50_cnt=coin_50_cnt+1
			sum=sum+50
			cv2.putText(frame, '50yen', (x,y-2), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 1, cv2.LINE_AA)
		elif(prediction==4):
			coin_100_cnt=coin_100_cnt+1
			sum=sum+100
			cv2.putText(frame, '100yen', (x,y-2), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 1, cv2.LINE_AA)
		elif(prediction==5):
			coin_500_cnt=coin_500_cnt+1
			sum=sum+500
			cv2.putText(frame, '500yen', (x,y-2), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 1, cv2.LINE_AA)
	cv2.putText(frame, '1yen:{0} 5yen: {1} 10yen: {2} 50yen:{3} 100yen:{4} 500yen:{5} num: {6} sum: {7} '.format (coin_1_cnt,coin_5_cnt,coin_10_cnt,coin_50_cnt,coin_100_cnt,coin_500_cnt,coin_cnt,sum), (0, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1, cv2.LINE_AA)
	cv2.imshow('image',cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
	key=cv2.waitKey(1) & 0xFF
	if key == ord('q'):
		break
	
	

capture.release()
cv2.destroyAllWindows()

